In [2]:
import numpy as np
import common.midi as md

Overall Equation:

$$T(x, \verb|chord|) = (\alpha) \cdot T_{\text{global}}(x) + (1 - \alpha) \cdot T_{\text{local}}(\verb|chord|)$$

There are two components to the equation:
- Global tension: this is the base tension of the chord relative to the tonic.
- Local tension: this the tension of the chord inherent to the structure of the chord.

$$T_{\text{global}}(x)=0.02375x^5 - 0.467803x^4 + 3.416098x^3 - 11.37083x^2 + 16.9965x - 8.598$$

$$T_{\text{local}}(\verb|chord|) = \sqrt{\frac{1}{N}\sum_{i \in \verb|chord|}^{N}{\left(\verb|tension|^{\{ i \}}\right)^2}}$$


In [3]:
def t_global(x, zero_indexed=False):
    x += 1 if zero_indexed else 0
    if not (1 <= x <= 7):
        raise ValueError('Outside of bounds 1-7')
    return round((0.02375 * x ** 5 - 0.467803 * x ** 4 + 3.416098 * x ** 3 - 11.37083 * x ** 2 + 16.9965 * x - 8.598), 3)

t_global(5)

0.968

In [4]:
intervals = {
    0: 0.0,
    1: 0.9,
    2: 0.8,
    3: 0.25,
    4: 0.15,
    5: 0.1,
    6: 1.0,
    7: 0.05,
    8: 0.5,
    9: 0.4,
    10: 0.6,
    11: 0.7
}

def t_local(chord):

    chord = [note % 12 for note in chord]

    tensions = []
    for i in range(1, len(chord)):
        interval1 = abs(chord[i] - chord[0]) # Total
        interval2 = abs(chord[i] - chord[i - 1]) # In between
        tensions.append(intervals[interval1])
        # tensions.append(intervals[interval2])
    # print(tensions)

    if tensions:
        rms = np.sqrt(np.mean(np.square(tensions))) / 0.8
    else:
        rms = 0.0

    rms = min(rms, 1.0)
    return round(float(rms), 3)

t_local([0, 3, 6])

0.911

In [5]:
def tension(root, chord, alpha=0.5):
    return round(alpha * t_global(root) + (1 - alpha) * t_local(chord), 4)

In [6]:
tension(5, [0, 4, 7, 10], 0.6)

0.76

In [8]:
CHORDS = set()

maj = set(md.SCALES['major'])
for degree in md.SCALES['major']:
    note = md.int_to_note(degree)[:1]
    for harmony, values in md.HARMONIES_SHORT.items():
        curr = [degree + value for value in values]
        for val in curr:
            if val % 12 not in maj:
                curr = None
                break
        if curr:
            CHORDS.add(f'{note}_{harmony}')

CHORDS


{'A_dom7aug5',
 'A_dom7sus4',
 'A_m',
 'A_m7',
 'A_m9',
 'A_sus2',
 'A_sus4',
 'B_dim',
 'B_dom7aug5',
 'B_m7b5',
 'C_M',
 'C_M6',
 'C_M7',
 'C_M9',
 'C_sus2',
 'C_sus4',
 'D_dom7sus4',
 'D_m',
 'D_m7',
 'D_m9',
 'D_sus2',
 'D_sus4',
 'E_dom7aug5',
 'E_dom7sus4',
 'E_m',
 'E_m7',
 'E_sus4',
 'F_M',
 'F_M6',
 'F_M7',
 'F_M9',
 'F_sus2',
 'G_M',
 'G_M6',
 'G_dom7',
 'G_dom7sus4',
 'G_sus2',
 'G_sus4'}

In [10]:
res = {}

for chord in CHORDS:
    root, harmony = chord.split('_')

    rt = md.get_key_ints('C', 'major').index(md.note_to_int(root + '0')) + 1
    ch = md.HARMONIES_SHORT[harmony]
    res[chord] = tension(rt, ch, 0.7)

res

{'G_dom7': 0.812,
 'E_dom7aug5': 0.288,
 'C_M7': 0.1554,
 'E_m7': 0.2586,
 'C_M': 0.0342,
 'E_sus4': 0.1419,
 'A_m7': 0.3622,
 'D_dom7sus4': 0.4932,
 'D_m7': 0.5022,
 'G_M6': 0.7583,
 'B_m7b5': 0.8883,
 'G_M': 0.7118,
 'B_dim': 0.8532,
 'C_sus2': 0.1734,
 'F_M9': 0.6545,
 'G_dom7sus4': 0.8096,
 'A_m9': 0.4147,
 'F_sus2': 0.6263,
 'A_sus2': 0.3946,
 'A_dom7aug5': 0.3916,
 'D_sus4': 0.3855,
 'F_M7': 0.6083,
 'G_sus4': 0.7019,
 'D_m': 0.4164,
 'E_dom7sus4': 0.2496,
 'A_dom7sus4': 0.3532,
 'C_sus4': 0.0243,
 'C_M6': 0.0807,
 'G_sus2': 0.851,
 'A_sus4': 0.2455,
 'C_M9': 0.2016,
 'B_dom7aug5': 0.8004,
 'F_M6': 0.5336,
 'E_m': 0.1728,
 'F_M': 0.4871,
 'D_sus2': 0.5346,
 'D_m9': 0.5547,
 'A_m': 0.2764}